In [39]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-07-05 08:19:38--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-07-05 08:19:38 (56.3 MB/s) - ‘minsearch.py.1’ saved [4073/4073]



In [40]:
import minsearch

/workspaces/llm-zoomcamp/01-Intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [41]:
import json

In [42]:
import openai
from openai import OpenAI

In [43]:
import os
from dotenv import load_dotenv

In [44]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [45]:
client = OpenAI()

In [46]:
with open('../documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [47]:
documents = []

for course_dict in docs_raw:
    for docs in course_dict['documents']:
        docs['course'] = course_dict['course']
        documents.append(docs)

In [48]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [49]:
index = minsearch.Index(
    text_fields = ['text', 'section', 'question'],
    keyword_fields = ['course']
)

In [50]:
index.fit(documents)

In [51]:
q = 'the course has already started, can I still enroll?'

In [52]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [53]:
def build_prompt(query, search_results):
    prompt_template = """
        You are course teaching assistant, Answer the QUESTION  based on CONTEXT from FAQ database.
        Use only facts from teh CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE

        QUESTION: {question}

        CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [54]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [78]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [79]:
query = "the course has already started, can I still enroll?"
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

## Search with Elastic Search

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200')

In [35]:
es_client.info()

ObjectApiResponse({'name': '695702293d30', 'cluster_name': 'docker-cluster', 'cluster_uuid': '7CiLm6SgR0uZP2YBVeR8Og', 'version': {'number': '9.0.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'cc7302afc8499e83262ba2ceaa96451681f0609d', 'build_date': '2025-06-18T22:09:56.772581489Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [55]:
# progress bar
from tqdm.auto import tqdm

In [56]:
# Loop melalui setiap dokumen dalam list 'documents' dengan progress bar dari tqdm
# Setiap dokumen diindeks (disimpan) ke Elasticsearch pada indeks bernama 'index_name'
# Fungsi 'index()' mengirim data ke Elasticsearch agar bisa dicari dan dianalisis

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 251.92it/s]


In [58]:
query = "the course has already started, can I still enroll?"

In [65]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    # search question
    response = es_client.search(index=index_name, body=search_query)

    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [67]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag(query)